In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import time



In [2]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),  # Adjust to the input size expected by ResNet-150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 358 images belonging to 4 classes.
Found 86 images belonging to 4 classes.


In [3]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Four classes

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Initialize variables to keep track of time and accuracy
total_time = 0
total_accuracy = 0
epochs=12
# Training loop
for epoch in range(epochs):
    start_time = time.time()  # Record the start time for this epoch
    
    # Train the model for one epoch
    history = model.fit(train_generator, epochs=1, validation_data=validation_generator)
    
    end_time = time.time()  # Record the end time for this epoch
    epoch_time = end_time - start_time  # Calculate epoch time
    
    # Update total time and accuracy
    total_time += epoch_time
    total_accuracy += history.history['accuracy'][0]  # Assuming 'accuracy' is in history
    
    # Print epoch statistics
    print(f"Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f}s - Accuracy: {history.history['accuracy'][0]:.4f}")

# Calculate and display average time and accuracy
average_time = total_time / epochs
average_accuracy = total_accuracy / epochs
print(f"Average Time: {average_time:.2f}s")
print(f"Accuracy: {average_accuracy:.4f}")


12/12 [==============================] - 425s 32s/step - loss: 1.4363 - accuracy: 0.4190 - val_loss: 25.6425 - val_accuracy: 0.3721
Epoch 1/12 - Time: 426.74s - Accuracy: 0.4190
12/12 [==============================] - 425s 35s/step - loss: 1.0135 - accuracy: 0.5615 - val_loss: 123.1002 - val_accuracy: 0.2326
Epoch 2/12 - Time: 426.55s - Accuracy: 0.5615
12/12 [==============================] - 478s 40s/step - loss: 0.9167 - accuracy: 0.6257 - val_loss: 7584.8960 - val_accuracy: 0.2326
Epoch 3/12 - Time: 479.99s - Accuracy: 0.6257
12/12 [==============================] - 445s 37s/step - loss: 0.7467 - accuracy: 0.7039 - val_loss: 558.3105 - val_accuracy: 0.1628
Epoch 4/12 - Time: 447.12s - Accuracy: 0.7039
12/12 [==============================] - 437s 36s/step - loss: 0.7815 - accuracy: 0.6760 - val_loss: 5926.8389 - val_accuracy: 0.2326
Epoch 5/12 - Time: 438.03s - Accuracy: 0.6760
12/12 [==============================] - 451s 38s/step - loss: 0.8100 - accuracy: 0.6872 - val_loss: 17.

In [7]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)


Found 444 images belonging to 4 classes.
14/14 [==============================] - 89s 6s/step - loss: 53.4120 - accuracy: 0.3626


In [8]:
import tensorflow_model_optimization as tfmot

# Annotate the model for quantization-aware training
quant_annotate_model = tfmot.quantization.keras.quantize_annotate_model(model)


In [9]:
# Compile the quantization-aware model
quant_annotate_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Train the quantization-aware model

# Initialize variables to keep track of time and accuracy
total_time = 0
total_accuracy = 0
epochs=12
# Training loop
for epoch in range(epochs):
    start_time = time.time()  # Record the start time for this epoch
    
    # Train the model for one epoch
    history = quant_annotate_model.fit(train_generator, epochs=1, validation_data=validation_generator)
    
    end_time = time.time()  # Record the end time for this epoch
    epoch_time = end_time - start_time  # Calculate epoch time
    
    # Update total time and accuracy
    total_time += epoch_time
    total_accuracy += history.history['accuracy'][0]  # Assuming 'accuracy' is in history
    
    # Print epoch statistics
    print(f"Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f}s - Accuracy: {history.history['accuracy'][0]:.4f}")

# Calculate and display average time and accuracy
average_time = total_time / epochs
average_accuracy = total_accuracy / epochs
print(f"Average Time: {average_time:.2f}s")
print(f"Accuracy: {average_accuracy:.4f}")


12/12 [==============================] - 488s 36s/step - loss: 0.8748 - accuracy: 0.6592 - val_loss: 182.8258 - val_accuracy: 0.2326
Epoch 1/12 - Time: 489.35s - Accuracy: 0.6592
12/12 [==============================] - 445s 37s/step - loss: 0.7693 - accuracy: 0.6788 - val_loss: 243.2834 - val_accuracy: 0.2558
Epoch 2/12 - Time: 446.40s - Accuracy: 0.6788
12/12 [==============================] - 439s 37s/step - loss: 0.5750 - accuracy: 0.7654 - val_loss: 691.7939 - val_accuracy: 0.2326
Epoch 3/12 - Time: 440.94s - Accuracy: 0.7654
12/12 [==============================] - 439s 37s/step - loss: 0.6406 - accuracy: 0.7458 - val_loss: 1905.6101 - val_accuracy: 0.2326
Epoch 4/12 - Time: 441.28s - Accuracy: 0.7458
12/12 [==============================] - 446s 37s/step - loss: 0.6088 - accuracy: 0.7514 - val_loss: 598.9529 - val_accuracy: 0.2442
Epoch 5/12 - Time: 447.28s - Accuracy: 0.7514
12/12 [==============================] - 446s 37s/step - loss: 0.5663 - accuracy: 0.7682 - val_loss: 730